In [2]:
import os,sys #user to create and modify file name and save it
from netCDF4 import Dataset#to read the .nc files
import pandas as pd #for dataframe activities
from datetime import datetime,timedelta #for the time related activities
import matplotlib.pyplot as plt #for the Ploting purpose
import numpy as np #numerical fucntions
import datetime 
import warnings #to ignore unnecessary warnings
import xarray as xr
import numpy as np
import regionmask
import geopandas as gp
import glob                                                                 
from pyhdf.SD import SD, SDC
warnings.filterwarnings('ignore')

# Correction and clipping of RH

In [4]:
os.chdir('/Volumes/PtatoBasket/Dataset_New/ERA-5 RH/PreMon')
ds=xr.open_dataset('RH-MAM_daily.nc')


In [91]:
from datetime import datetime,timedelta,date
RH=ds.r
time,pressure,lat,lon=RH.indexes.values()
x=list(time)
y=[]

for n in x:
    y.append(n.date())

In [92]:
import os
from datetime import datetime,timedelta,date
date_list=[]
for yr in range(2005,2022):
    month=[3,4,5]
    for mon in month:
        for day in range(1,32):
            try:
                d=date(yr,mon,day)
                delta=timedelta(days=1)
                year=d.year
                doy=d.timetuple().tm_yday
                doy="{:03d}".format(doy)
                # date_list.append('A'+str(year)+str(doy)+'.hdf')#MODIS-GESDISC
                # date_list.append(d.strftime('%Y%m%d'))#IMGERG/PBL
                date_list.append(d)#IMGERG/PBLH
                d+=delta
            except ValueError:
                pass
              

In [93]:
  
missing_dates=list(set(y)-set(date_list))
indexes = [y.index(x) for x in missing_dates]
unwanted_times=[time[x] for x in indexes]


In [94]:

for x in unwanted_times:
    # print(x)
    RH=RH.isel(time=RH['time']!=x)
    
RH=RH.mean(dim = 'level')

In [95]:

#Clipping to shape File and Creating Mask File
shapefile ="/Volumes/ACIML/Main/SHPs/BOB_OCEAN/BOB_OCEAN.shp"
countries=gp.read_file(shapefile,engine='pyogrio')
c_list=list(countries['featurecla'])
c_list_unique=set(list(countries['featurecla']))
indexes=[c_list.index(x) for x in c_list_unique]
countries_mask_poly=regionmask.Regions(outlines=countries.geometry[indexes],name='featurecla',numbers=indexes,names=countries.featurecla[indexes])
print('It is Clipping')
mask=countries_mask_poly.mask(RH,lat_name='latitude',lon_name='longitude')
# mask.to_netcdf('BOB_MASK.nc')

# mask=xr.open_dataarray('/Volumes/ACIML/Main/SHPs/BOB_MASK.nc')
masked_shape=RH.where(mask==0)
min_lon = 78.00
max_lon = 96.00
min_lat = 8.00
max_lat = 23.00

cropped_ds = masked_shape.sel(latitude=slice(max_lat,min_lat), longitude=slice(min_lon,max_lon))


It is Clipping


In [97]:
cropped_ds.to_netcdf('PreMonDaily_RHavg.nc')

# For IMGERG Precipitation

In [ ]:
st=time.time()
os.chdir('/Volumes/PtatoBasket/Dataset_New/IMERG')
xxx=[]
a=[]
for file in list(glob.glob('PreMon/*.nc4')):

    ds=xr.open_dataset(file)

    # Retrieve attributes.
    attrs = ds.attrs
    dat=attrs["BeginDate"]
    dat=datetime.strptime(dat,'%Y-%m-%d')
    # January 1st, 1900
    start_datetime = datetime(1900, 1, 1)

    # Calculate the time delta (difference) in hours
    delta = (dat - start_datetime).total_seconds() / 3600


    xxx.append(delta)


In [7]:
from datetime import datetime,timedelta
arr=str('A2005001.hdf').split('.')
year=str(arr[0])[1:5]
days=str(arr[0])[5:]
dat=datetime(int(year), 1, 1) + timedelta(int(days) - 1)
# dat=(dat-datetime(2005,1,1)).days
# January 1st, 1900
start_datetime = datetime(1900, 1, 1)

# Calculate the time delta (difference) in hours
delta = (dat - start_datetime).total_seconds() / 3600


In [1]:
from pyhdf.SD import SD, SDC
import glob
os.chdir('/Volumes/PtatoBasket/Dataset_New/PrecipitableWater-MODIS/PreMon')
x=list(glob.glob('A*.hdf'))
for file in list(glob.glob('*.hdf')):

    reader = open(file)
    hdf = SD(file, SDC.READ)


RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

SystemError: initialization of _hdfext raised unreported exception